# Sprint ディープラーニングフレームワーク1

In [2]:
import tensorflow as tf

import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# 【問題1】スクラッチを振り返る
<pre>
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。

（例）

重みを初期化する必要があった
エポックのループが必要だった
それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。

重みを初期化する必要があった、その際shapeを合わせる必要があった  
エポックのループが必要だった  
最適化手法を指定する必要があった  
活性化関数を指定する必要があった  
ノード数を指定する必要があった  

# 【問題2】スクラッチとTensorFlowの対応を考える
<pre>
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。

それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。

重みはVariableでshapeと初期値をどのように設定するかを決め、初期化の処理　tf.global_variables_initializer()で全て同時に初期化している  
エポックのループは計算グラフを実行する際に決めた値を行なっている  
最適化手法は計算グラフを実行する前に変数に代入し、代入したものに処理を追加したものを別の変数に代入している。計算グラフを実行する際はこれを実行する。  
ニューラルネットワークの間に挟む活性化関数はニューラルネットワークの中で宣言している。  
ノード数はハイパーパラメータとして先に設定していた  

In [6]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""

# データセットの読み込み
df = pd.read_csv("/Users/morishuuya/Desktop/dataset/DIC/iris-species/Iris.csv")
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 0.0000, val_loss : 3.4371, acc : 1.000, val_acc : 0.750
Epoch 1, loss : 0.0000, val_loss : 0.5258, acc : 1.000, val_acc : 0.875
Epoch 2, loss : 3.8298, val_loss : 10.4621, acc : 0.500, val_acc : 0.688
Epoch 3, loss : 0.0000, val_loss : 1.7056, acc : 1.000, val_acc : 0.812
Epoch 4, loss : 0.0000, val_loss : 4.8342, acc : 1.000, val_acc : 0.750
Epoch 5, loss : 0.0000, val_loss : 2.8994, acc : 1.000, val_acc : 0.750
Epoch 6, loss : 1.7623, val_loss : 9.7120, acc : 0.500, val_acc : 0.688
Epoch 7, loss : 0.0000, val_loss : 3.8396, acc : 1.000, val_acc : 0.688
Epoch 8, loss : 0.0000, val_loss : 2.4149, acc : 1.000, val_acc : 0.875
Epoch 9, loss : 0.0000, val_loss : 0.9010, acc : 1.000, val_acc : 0.938
test_acc : 0.800


# 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
<pre>
Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。

Iris Species

2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。

In [7]:
df = pd.read_csv("/Users/morishuuya/Desktop/dataset/DIC/iris-species/Iris.csv")
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y[y=="Iris-setosa"] = 2
y = y.astype(np.int)[:, np.newaxis]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
y_train = enc.fit_transform(y_train)
y_test = enc.fit_transform(y_test)
y_val = enc.fit_transform(y_val)

In [9]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(tf.nn.softmax(logits), 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 2.3969, val_loss : 9.8918, acc : 0.667, val_acc : 0.708
Epoch 1, loss : 0.9530, val_loss : 4.0192, acc : 0.833, val_acc : 0.708
Epoch 2, loss : 0.0000, val_loss : 4.2052, acc : 1.000, val_acc : 0.667
Epoch 3, loss : 0.0328, val_loss : 6.6991, acc : 1.000, val_acc : 0.792
Epoch 4, loss : 0.0000, val_loss : 2.5844, acc : 1.000, val_acc : 0.792
Epoch 5, loss : 0.0000, val_loss : 2.1838, acc : 1.000, val_acc : 0.833
Epoch 6, loss : 0.0000, val_loss : 4.4951, acc : 1.000, val_acc : 0.917
Epoch 7, loss : 0.0000, val_loss : 2.9667, acc : 1.000, val_acc : 0.917
Epoch 8, loss : 0.0000, val_loss : 2.1708, acc : 1.000, val_acc : 0.875
Epoch 9, loss : 0.0000, val_loss : 3.1697, acc : 1.000, val_acc : 0.917
test_acc : 0.967


## 変更点
**n_class:**  
2  →　3

**loss_op:**  
sigmoid→softmax  
  
**correct_pred:**  
sigmoid→softmax  
sign→argmax  

# 【問題4】House Pricesのモデルを作成
<pre>
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。

House Prices: Advanced Regression Techniques

この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。

分類問題と回帰問題の違いを考慮してください。

In [23]:
df = pd.read_csv("/Users/morishuuya/Desktop/dataset/kaggle/HousePrice/train.csv")
X = df.loc[:, ["GrLivArea", "YearBuilt"]].values
y = df["SalePrice"].values
y = np.log(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [26]:
# hyper params
learning_rate = 0.1
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None])

mini_batch_get = GetMiniBatch(X_train, y_train, batch_size = batch_size)

def simple_net(x):
    weights={
        "W1":tf.Variable(tf.random_normal([n_input, n_hidden1])),
        "W2":tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        "W3":tf.Variable(tf.random_normal([n_hidden2, n_classes]))}
    
    biases={
        "b1":tf.Variable(tf.random_normal([n_hidden1])),
        "b2":tf.Variable(tf.random_normal([n_hidden2])),
        "b3":tf.Variable(tf.random_normal([n_classes]))
    }
    
    layer1 = tf.add(tf.matmul(x, weights["W1"]) , biases["b1"])
    layer1 = tf.nn.relu(layer1)
    layer2 = tf.add(tf.matmul(layer1, weights["W2"]), biases["b2"])
    layer2 = tf.nn.relu(layer2)
    output = tf.matmul(layer2, weights["W3"]) + biases["b3"]
    return output

# load net
logits = simple_net(X)
# loss function

loss_op = tf.reduce_mean(tf.square(Y - logits))
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

#rmse
rmse = tf.sqrt(tf.reduce_mean(tf.square(Y - logits)))
#r2score
r_2 = 1 - ((tf.reduce_sum(tf.square(logits - Y))) / (tf.reduce_sum(tf.square(logits - tf.reduce_mean(Y)))))

# initializer
init = tf.global_variables_initializer()

# action
with tf.Session() as sess:
    sess.run(init)
    for epochs in range(num_epochs):
        total_loss = 0
        total_acc = 0
        for i , (X_train_mini, y_train_mini) in enumerate(mini_batch_get):
            sess.run(train_op, feed_dict={X:X_train_mini, Y:y_train_mini})
            loss  = sess.run(loss_op , feed_dict={X:X_train_mini, Y:y_train_mini})
            total_loss += loss
        total_loss /= n_samples
        val_loss = sess.run(loss_op,  feed_dict={X:X_val, Y:y_val})
        print("epochs:", epochs, "total_loss:", total_loss.round(), "val_loss:", val_loss)
        print("MSE:", sess.run(rmse, feed_dict={X:X_val, Y:y_val}))
        print("-"*50)
        

epochs: 0 total_loss: 64457853.0 val_loss: 163788.14
MSE: 404.7075
--------------------------------------------------
epochs: 1 total_loss: 8372.0 val_loss: 40115.953
MSE: 200.28967
--------------------------------------------------
epochs: 2 total_loss: 3868.0 val_loss: 40373.484
MSE: 200.93155
--------------------------------------------------
epochs: 3 total_loss: 3064.0 val_loss: 26956.352
MSE: 164.1839
--------------------------------------------------
epochs: 4 total_loss: 2408.0 val_loss: 16797.969
MSE: 129.60698
--------------------------------------------------
epochs: 5 total_loss: 1886.0 val_loss: 11653.872
MSE: 107.9531
--------------------------------------------------
epochs: 6 total_loss: 1490.0 val_loss: 9561.184
MSE: 97.7813
--------------------------------------------------
epochs: 7 total_loss: 1147.0 val_loss: 6738.1553
MSE: 82.086266
--------------------------------------------------
epochs: 8 total_loss: 885.0 val_loss: 4312.8613
MSE: 65.67238
--------------------

## 変更点
**n_class:**  
2  →　1

**Yの次元:**  
2 →1

**loss_op:**  
sigmoid→MSE  
  
**結果の精度予測**  
accuracy →　RMSE or R2_score

# 【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。

3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。

スクラッチで実装したモデルの再現を目指してください。

In [20]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train  = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.fit_transform(y_test[:,  np.newaxis])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2, random_state=0)

Using TensorFlow backend.


In [21]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(tf.nn.softmax(logits), 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 1.4040, val_loss : 1.0648, acc : 0.500, val_acc : 0.707
Epoch 1, loss : 0.4666, val_loss : 1.0201, acc : 0.900, val_acc : 0.787
Epoch 2, loss : 0.3083, val_loss : 0.5054, acc : 0.900, val_acc : 0.876
Epoch 3, loss : 0.2950, val_loss : 0.3588, acc : 0.900, val_acc : 0.913
Epoch 4, loss : 0.1135, val_loss : 0.3088, acc : 1.000, val_acc : 0.925
Epoch 5, loss : 0.0917, val_loss : 0.3445, acc : 1.000, val_acc : 0.919
Epoch 6, loss : 0.1481, val_loss : 0.2996, acc : 1.000, val_acc : 0.929
Epoch 7, loss : 0.3140, val_loss : 0.3293, acc : 0.900, val_acc : 0.928
Epoch 8, loss : 0.4386, val_loss : 0.3143, acc : 0.900, val_acc : 0.927
Epoch 9, loss : 0.3708, val_loss : 0.3732, acc : 0.800, val_acc : 0.926
test_acc : 0.925
